<a href="https://colab.research.google.com/github/rchicangana/AnalisisDatosProyecto/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1  Introduccion

El análisis exploratorio de datos es una etapa fundamental en cualquier proyecto de Inteligencia Artificial y Ciencia de Datos, puesto que permite entender las características y patrones en los datos, así como identificar relaciones entre variables.

A continuación se presenta un estudio basado en datos de consumo de energía eléctrica horaria registrados en Cali, Colombia, junto con las temperaturas correspondientes a cada hora. El objetivo principal es explorar la relación entre estas dos variables y, posteriormente, aplicar diferentes modelos para realizar el pronósticos de corto plazo del demanda de energía.

# 2  Contexto

La demanda de energía eléctrica en una ciudad se ve inlfuenciada por muchos factores, como la temperatura, factores sociales, cronograma de produccion de grandes clientes, entre otros.

En regiones con climas variables, como es el caso de muchas ciudades Colombianas, la demanda es afectada por el uso intensivo de sistemas de refrigeración en épocas calurosas.

Los datos utilizados en este análisis incluyen mediciones horarias del consumo de energía y la temperatura registrada en la misma franja horaria, tipos de dia, mes y año. Lo que permite explorar la correlación entre estas variables.

Adicionalmente, se busca evaluar el desempeño de diferentes modelos como regresiones lineales, redes neuronales y modelos basados en árboles de decisión, para predecir el consumo de energía en un horizonte de corto plazo.

## 2.1  Descripción de los campos

- FECHA: Fecha en formato ISO de la lectura del consumo
- TEMPERATURA: Temperatura en grados Centigrados al momento de la toma del dato
- CONSUMO: Variable objetvio medido en MVA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
import glob
import holidays
#importo el dataset Principal
df = pd.read_csv('Data/datasetEmcali.csv')



In [ ]:
print("primeras filas \n")
print(df.head())


print("se describen las columnas \n")
print(df.info())

print("dimensiones \n")
print(df.shape)

# 3  Análisis exploratorio


## 3.1  Análisis de estructura


Se realiza un analisis de la forma del dataset, se relacionan tipos de datos, estadisticas descriptivas y valores nulos.

Se adicionan columnas, basadas en la fecha, con la hipotesis que esto permitira al modelo entender e interpretar mejor los datos.

Tratamiento de dataset Humedad

In [ ]:
#Defino el formato de la fecha
df["FECHA"] = pd.to_datetime(df["FECHA"], format="mixed", errors="coerce")

# Cargar datasets de humedad
humedad_files = glob.glob("Data/reporte_sisaire-humedad-*.csv")
df_humedad_list = [pd.read_csv(file) for file in humedad_files]
df_humedad = pd.concat(df_humedad_list)

# Convertir fechas en el dataset de humedad
df_humedad["Fecha inicial"] = pd.to_datetime(df_humedad["Fecha inicial"], format="%Y-%m-%d %H:%M")

# Promediar la humedad por hora
df_humedad_avg = df_humedad.groupby("Fecha inicial")["HAire2"].mean().reset_index()
df_humedad_avg.rename(columns={"Fecha inicial": "FECHA", "HAire2": "HUMIDITY"}, inplace=True)

df_humedad_avg.head()

Tratamiento de dataset Temperatura

In [ ]:
# Cargar datasets de velocidad del viento
viento_files = glob.glob("Data/reporte_sisaire-vViento-*.csv")
df_viento_list = [pd.read_csv(file) for file in viento_files]
df_viento = pd.concat(df_viento_list)

# Convertir fechas en el dataset de viento
df_viento["Fecha inicial"] = pd.to_datetime(df_viento["Fecha inicial"], format="%Y-%m-%d %H:%M")

# Promediar la velocidad del viento por hora
df_viento_avg = df_viento.groupby("Fecha inicial")["VViento"].mean().reset_index()
df_viento_avg.rename(columns={"Fecha inicial": "FECHA", "VViento": "WIND"}, inplace=True)

df_viento_avg.head()

Union de los dataset

In [ ]:


# Unir los datos de humedad y viento con el dataset principal
df_humedad_avg["FECHA"] = pd.to_datetime(df_humedad_avg["FECHA"])
df_viento_avg["FECHA"] = pd.to_datetime(df_viento_avg["FECHA"])

df = df.merge(df_humedad_avg, on="FECHA", how="left", suffixes=("","_extra"))
df = df.merge(df_viento_avg, on="FECHA", how="left" , suffixes=("","_extra"))

# Combinar las columnas duplicadas en una sola, tomando el promedio de los valores
#df["HUMEDAD"] = df[["HUMEDAD", "HUMEDAD_extra"]].mean(axis=1)
#df["VIENTO"] = df[["VIENTO", "VIENTO_extra"]].mean(axis=1)

# Eliminar las columnas duplicadas
#df = df.drop(columns=[col for col in df.columns if col.endswith("_extra")])
df = df.drop(columns=[col for col in df.columns if col.endswith("_x")])
df = df.drop(columns=[col for col in df.columns if col.endswith("_y")])

print("primeras filas \n")
print(df.head())


print("se describen las columnas \n")
print(df.info())

print("dimensiones \n")
print(df.shape)

In [ ]:

# le doy formato a las columnas que va a ser numericas
df['CONSUMO'] = pd.to_numeric(df['CONSUMO'], errors='coerce')

df['TEMPERATURA'] = pd.to_numeric(df['TEMPERATURA'], errors='coerce')

df['HUMIDITY'] = pd.to_numeric(df['HUMIDITY'], errors='coerce')

df['WIND'] = pd.to_numeric(df['WIND'], errors='coerce')

# Elimino las filas que tienen valores nulos en 'FECHA' y 'CONSUMO', 'TEMPERATURA', 'HUMEDAD', 'VIENTO'

df = df.dropna(subset=['FECHA', 'CONSUMO', 'TEMPERATURA', 'HUMIDITY', 'WIND'])
    


def es_festivo_colombia(fecha):
    """Verifica si una fecha es festivo en Colombia."""
    co_holidays = holidays.CO(years=fecha.year)
    return fecha in co_holidays
df['Is_holiday'] = df['FECHA'].apply(es_festivo_colombia)

# Extract numerical features from 'FECHA'
df['Year'] = df['FECHA'].dt.year
df['Month'] = df['FECHA'].dt.month
df['Day'] = df['FECHA'].dt.day
df['Hour'] = df['FECHA'].dt.hour
df['Dayofweek'] = df['FECHA'].dt.dayofweek
df['Dayofyear'] = df['FECHA'].dt.dayofyear
df['Weekofyear'] = df['FECHA'].dt.isocalendar().week
df['Is_month_end'] = df['FECHA'].dt.is_month_end
df['Is_month_start'] = df['FECHA'].dt.is_month_start
df['Is_quarter_end'] = df['FECHA'].dt.is_quarter_end
df['Is_quarter_start'] = df['FECHA'].dt.is_quarter_start
df['Is_year_end'] = df['FECHA'].dt.is_year_end
df['Is_year_start'] = df['FECHA'].dt.is_year_start

# Eliminar la columna 'FECHA'
df = df.drop('FECHA', axis=1)

# Convert categorical columns to categorical
categorical_cols = ['Year', 'Month', 'Day', 'Hour', 'Dayofweek', 'Dayofyear', 'Weekofyear']
for col in categorical_cols:
    df[col] = df[col].astype('category')

print("Forma de lo datos:", df.shape)
print("\nTipo de datos por columnas:\n\n", df.dtypes)
print("\nEstadisticas descriptivas:\n\n", df.describe())
print("\nValores Faltantes:\n\n", df.isnull().sum())
print("\nValores Faltantes(%):\n\n", (df.isnull().sum() / len(df)) * 100)

se notan valores nulos

## 3.2  Análisis de variables numericas y continuas


In [ ]:

numerical_columns = ['CONSUMO', 'TEMPERATURA', 'HUMIDITY', 'WIND']

correlation_matrix = df[numerical_columns].corr()

plt.figure(figsize=(10, 8))

#Matriz de correlación del DataFrame con valores de correlación entre -1 y 1, utilizando una paleta de colores verde-amarillo-rojo para resaltar las relaciones positivas y negativas, respectivamente.
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', fmt="0.4f")

plt.title('Correlation Matrix of Numerical Columns')

plt.show()

continuous_cols = ['TEMPERATURA', 'CONSUMO', 'HUMIDITY', 'WIND']

num_cols = 2  # columns per row
num_rows = (len(continuous_cols) * 2 + num_cols - 1) // num_cols # Calculate the rows needed.

# Create subplots in a grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows))  # Adjust figsize as needed
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

for i, col in enumerate(continuous_cols):
    # Histogram
    axes[i * 2].hist(df[col], bins=20)
    axes[i * 2].set_title(f'Distribution of {col}')

    # Scatterplot
    sns.scatterplot(x=col, y='CONSUMO', data=df, ax=axes[i * 2 + 1])
    axes[i * 2 + 1].set_title(f'CONSUMO vs. {col}')

    correlation = df[col].corr(df['CONSUMO'])
    print(f"Correlation between {col} and CONSUMO: {correlation}")

# Remove any unused subplots
for j in range(len(continuous_cols) * 2, num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


## 3.3  Análisis de variables Categoricas


In [ ]:
# Assuming 'df' is your DataFrame and 'categorical_cols' is defined
#categorical_cols = ['Year', 'Month', 'Day', 'Hour', 'Dayofweek', 'Dayofyear', 'Weekofyear']

# Determine the number of rows needed for the subplots
num_cols = 4  # 4 columns per row
num_rows = (len(categorical_cols) * 2 + num_cols - 1) // num_cols # Calculate the rows needed.

# Create subplots in a grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows))  # Adjust figsize as needed
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

for i, col in enumerate(categorical_cols):
    # Countplot
    sns.countplot(x=col, data=df, ax=axes[i * 2])
    axes[i * 2].set_title(f'Frequency Distribution of {col}')
    axes[i * 2].tick_params(axis='x', rotation=45)  # Rotate x-axis labels if needed

    # Boxplot
    sns.boxplot(x=col, y='CONSUMO', data=df, ax=axes[i * 2 + 1])
    axes[i * 2 + 1].set_title(f'CONSUMO vs. {col}')
    axes[i * 2 + 1].tick_params(axis='x', rotation=45)  # Rotate x-axis labels if needed

# Remove any unused subplots
for j in range(len(categorical_cols) * 2, num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()  # Adjust subplot parameters to give specified padding.
plt.show()

Los años 2020 y 2021 presentaron un fenomeno social el cual no es posible modelar en este espacio. Las restricciones a la movilidad en la pandemia y el paro nacional, impactaron directamente el consumo de eneriga en la ciudad de Cali.

Se concidera prudente, sacar del estudio estos dos años.

In [ ]:
df = df.drop(df[(df['Year'] == 2020) | (df['Year'] == 2021)].index)
df = df.reset_index(drop=True)

num_cols = 4  # 4 columns per row
num_rows = (len(categorical_cols) * 2 + num_cols - 1) // num_cols # Calculate the rows needed.
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows))  # Adjust figsize as needed
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

for i, col in enumerate(categorical_cols):
    sns.countplot(x=col, data=df, ax=axes[i * 2])
    axes[i * 2].set_title(f'Frequency Distribution of {col}')
    axes[i * 2].tick_params(axis='x', rotation=45)  # Rotate x-axis labels if needed

    sns.boxplot(x=col, y='CONSUMO', data=df, ax=axes[i * 2 + 1])
    axes[i * 2 + 1].set_title(f'CONSUMO vs. {col}')
    axes[i * 2 + 1].tick_params(axis='x', rotation=45)  # Rotate x-axis labels if needed
for j in range(len(categorical_cols) * 2, num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()  # Adjust subplot parameters to give specified padding.
plt.show()

### 3.4.1  Analisis de outliers


In [ ]:
# Columnas numéricas
numerical_cols = ['TEMPERATURA', 'CONSUMO', 'HUMIDITY', 'WIND']
# Box plots to visualize outliers
plt.figure(figsize=(12, 6))
for i, col in enumerate(numerical_cols):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(y=df[col])
    plt.title(f'Box Plot of {col}')
plt.tight_layout()
plt.show()

# Descriptive statistics and IQR outlier detection
for col in numerical_cols:
    print(f"\nAnalysis for numerical variable: {col}")
    print(df[col].describe())

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Number of outliers: {len(outliers)}")
    print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    # Document findings on potential causes and consequences of outliers (example)
    if col == 'TEMPERATURA':
        print("Potential causes for outliers in 'TEMPERATURA': Sensor malfunction, extreme weather events")
        print("Consequences: Skewed temperature distributions, inaccurate consumption predictions")

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=col, y='CONSUMO', data=df)
    plt.title(f'CONSUMO vs. {col} with Outliers')

    # Highlight outliers on the scatter plot
    if not outliers.empty:
        plt.scatter(outliers[col], outliers['CONSUMO'], color='red', label='Outliers')
        plt.legend()

    plt.show()

### 3.4.2  Conclusiones Outliers


- En el contexto energetico y de negocion, los bajos consumos se puden considedar como outliers

In [ ]:
df.isnull().sum()

# 4  Limpieza de datos


## 4.2  Imputación


In [ ]:
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
df_cleaned = df.copy()

# Crea series desplazadas para el valor anterior y siguiente
temperatura_anterior = df_cleaned['TEMPERATURA'].shift(1)
temperatura_siguiente = df_cleaned['TEMPERATURA'].shift(-1)

# Calcula el promedio del valor anterior y siguiente
promedio_anterior_siguiente = (temperatura_anterior + temperatura_siguiente) / 2

# Imputa los valores faltantes con el promedio
df_cleaned['TEMPERATURA'] = df_cleaned['TEMPERATURA'].fillna(promedio_anterior_siguiente)

# Manejar los valores NaN restantes al inicio o al final del DataFrame
df_cleaned['TEMPERATURA'] = df_cleaned['TEMPERATURA'].fillna(df_cleaned['TEMPERATURA'].median()) # Corregido

# Reassign the cleaned DataFrame to df
df = df_cleaned

print(df.head())
print(df.info())
print(df.isnull().sum())

- se contempla el proceso debido a la posibilidad de la llegada de nuevos datos, a travez del cual se pueden imputan lo nuevos datos nulos. el proceso de imputacion se esat llevando a cabo teniendo en cuenta la medida anterior y posterior con referencia al dato nulo, se realiza un promedio de estas dos medidas debido a que las variaciones de la temperatura en el rango de 1 hora es minimo.
- ademas se elimina la fecha porque ya no es relevante

## 4.1  Division de datasets


In [ ]:
y = df['CONSUMO']
X = df.drop(['CONSUMO'], axis=1)

print("primeras filas \n")
print(X.head())

print("se describen las columnas \n")
print(X.info())

print("dimensiones \n")
print(X.shape)
print("dimensiones \n")
print(y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 4.2  Estandarización y Escalamiento


Teniendo en cuenta que solo la variable temperatura es numerica y continua no se requiere Escalar o normalizar.

In [ ]:
# from sklearn.preprocessing import StandardScaler


# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


# 5  Feature Engineering


## 5.1  Dummificar variables


In [ ]:
# Use pd.get_dummies with the 'columns' parameter to specify which columns to dummify

print(X_train.head())
X_train = pd.get_dummies(X_train, columns=categorical_cols)
X_test = pd.get_dummies(X_test, columns=categorical_cols)

# Check if the number of columns in X_train and X_test are equal
print(f"Number of columns in X_train: {X_train.shape[1]} \n")
print(f"Number of columns in X_test: {X_test.shape[1]}")


## 5.2  PCA


In [ ]:
# Split data into training and testing/validation sets
X_train_final, X_test_val, y_train_final, y_test_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=42
)

# # Split testing/validation set into testing and validation sets
X_test, X_val, y_test, y_val = train_test_split(
    X_test_val, y_test_val, test_size=0.5, random_state=42
)

print(f"X_train_final shape: {X_train_final.shape}, y_train_final shape: {y_train_final.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


In [ ]:

# Initialize PCA with 95% variance retained
pca = PCA(n_components=0.95)

# Fit PCA on the training data
pca.fit(X_train_final)

# Transform the training, testing, and validation data
X_train_pca = pca.transform(X_train_final)
X_test_pca = pca.transform(X_test)
X_val_pca = pca.transform(X_val)

# Print the explained variance ratio
print(pca.explained_variance_ratio_)

# Replace original data with PCA-transformed data
X_train_final = X_train_pca
X_test = X_test_pca
X_val = X_val_pca

## 5.3  LDA


## 5.4  Clustering


No aplica, Colocar Explicacion

# 6 Data Pipeline


In [ ]:
# scoring data
from sklearn.metrics import accuracy_score

# importing pipes for making the Pipe flow
from sklearn.pipeline import Pipeline

#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipe = Pipeline([
  ('Estandarizacion', StandardScaler()),
  ('model', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),)
],verbose = True)

pipe.fit(X_train, y_train)

# y_pred = pipe.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy_score(y_test, pipe.predict(X_test)))

# 7  Modelamiento

RegresionLineal

In [ ]:
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

# Calculate MAE and MedAE
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Randomforest

In [ ]:
# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

# Train the model
rf_model.fit(X_train_final, y_train_final)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Print the evaluation metrics
print(f"Random Forest - Mean Squared Error (MSE): {mse_rf}")
print(f"Random Forest - R-squared (R2): {r2_rf}")

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_rf)

# Calculate the Median Absolute Error (MedAE)
medae = median_absolute_error(y_test, y_pred_rf)

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))

# Print the calculated MAE, MedAE, and RMSE values.
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Hiperparametros

In [ ]:
# Define la cuadrícula de hiperparámetros para buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
}

# Crea un objeto GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Ajusta el GridSearchCV a los datos de entrenamiento
grid_search.fit(X_train_final, y_train_final)

# Obtén el mejor modelo
best_model = grid_search.best_estimator_

# Evalúa el mejor modelo en los datos de prueba
y_pred_best = best_model.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

# Imprime las métricas de evaluación
print(f"Mejor modelo - Error cuadrático medio (MSE): {mse_best}")
print(f"Mejor modelo - R-cuadrado (R2): {r2_best}")

Red neuronal

In [ ]:
# 1. Definir el modelo de la red neuronal
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)  # Capa de salida para regresión
])

# 2. Compilar el modelo
model.compile(optimizer='adam', loss='mse')  # Usamos 'mse' (error cuadrático medio) para regresión

# 3. Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# 4. Realizar predicciones
y_pred = model.predict(X_test)

# 5. Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

# Opcional: Graficar la pérdida durante el entrenamiento
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Red neuronal con LSTM

In [ ]:
df.head()


In [ ]:
# 1. Preparar datos para LSTM (reshape a [muestras, pasos de tiempo, características])
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# 2. Definir el modelo LSTM
model = keras.Sequential([
    keras.layers.LSTM(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    keras.layers.Dense(1)
])

# 3. Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='mse')
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, verbose=1)

# 4. Realizar predicciones
y_pred = model.predict(X_test_reshaped)

# 5. Evaluar el modelo
# (calcula métricas como MSE, MAE, etc.)